In [ ]:

predict_data #raw data for new devs coming from 1.1

predictors = ['acc_lci_score', 'seniority_score', 'num_challenges', 'passed_num_challenges','total_problems','attempted_problems','num_correct','mean_dev_percentile','num_passed_stack', 'num_passed_skill', 'characters_in_reume','years_of_experience','top_skill_supply','top_skill_demand', 'top_stack_supply', 'correct_per_challenge', 'correct_per_questions', 'english_communication','quiz_answer']
target     = ['paying_cust'] 
dev_id     = ['dev_id']


ml_data_v2 = predict_data[predictors+target+dev_id]
num_cols   = [col for col in ml_data_v2.columns if ml_data_v2[col].dtype!='object']
obj_cols   = [col for col in ml_data_v2.columns if ml_data_v2[col].dtype=='object']
X      = ml_data_v2[predictors].copy()
Y      = ml_data_v2[target]
dev_id = ml_data_v2[dev_id]


for col in (obj_cols):
    X[col] = X[col].astype('category')  # or use map may be if we get dict
    X[col] = X[col].cat.codes



#global_means = X.mean()
#global_std = dict(X.std())
#global_vars = {'global_means' : global_means, 'global_std' : global_std}
#global_vars = {'global_means' : global_means, 'global_std' : global_std}
#pickle.dump(global_vars, open('global_vars_2020-08-05.sav', 'wb'))

global_vars = pickle.load(open('global_vars_2020-08-05.sav', 'rb'))
global_means = global_vars['global_means']
global_std = global_vars['global_std']

for i in X.columns:
    X[i] = (X[i]-global_means[i])/global_std[i]
    
norm_data = pd.concat([dev_id, Y, X],axis=1)
# on norm_data

a = [re.search(r'\d{2}-\d{2}-\d{2}', x).group(0) for x in glob.glob("../models/1.2-knnmputer-model*")]
b = sorted([dt.datetime.strptime(x,"%d-%m-%y") for x in a])
knnimpu_path=[val for val in glob.glob("../models/1.2-knnmputer-model*") if re.match('.*' + b[-1].strftime('%d-%m-%y'),val)][0]
loaded_model = pickle.load(open(knnimpu_path, 'rb'))
X_final = pd.DataFrame(loaded_model.transform(norm_predict_data[predictors].to_numpy()))
X_final.columns = predictors